# NVIDIA TensorRT-LLMを使用したOpenAI GPT-OSSモデルの最適化

このノートブックは、高性能推論のためにNVIDIAのTensorRT-LLMを使用して`gpt-oss`モデルを最適化する方法について、ステップバイステップのガイドを提供します。TensorRT-LLMは、Large Language Models（LLM）を定義し、NVIDIA GPU上で効率的に推論を実行するための最先端の最適化をサポートする、使いやすいPython APIをユーザーに提供します。TensorRT-LLMには、パフォーマンスの高い方法で推論実行を調整するPythonおよびC++ランタイムを作成するコンポーネントも含まれています。

TensorRT-LLMは以下の両方のモデルをサポートしています：
- `gpt-oss-20b`
- `gpt-oss-120b`

このガイドでは、`gpt-oss-20b`を実行します。より大きなモデルを試したい場合や、より多くのカスタマイゼーションが必要な場合は、[この](https://github.com/NVIDIA/TensorRT-LLM/blob/main/docs/source/blogs/tech_blog/blog9_Deploying_GPT_OSS_on_TRTLLM.md)デプロイメントガイドを参照してください。

注意：モデルが適切に動作するためには、入力プロンプトで[harmony response](http://cookbook.openai.com/articles/openai-harmony)形式を使用する必要がありますが、このガイドでは必須ではありません。

#### NVIDIA Brevでの起動
[NVIDIA Brev](https://developer.nvidia.com/brev)を使用することで、環境設定を簡素化できます。下のボタンをクリックして、必要な依存関係が事前に設定されたBrevインスタンス上でこのプロジェクトを起動してください。

デプロイが完了したら、「Open Notebook」ボタンをクリックして、このガイドを開始してください。

[![Launch on Brev](https://brev-assets.s3.us-west-1.amazonaws.com/nv-lb-dark.svg)](https://brev.nvidia.com/launchable/deploy?launchableID=env-30i1YjHsRWT109HL6eYxLUeHIwF)

## 前提条件

### ハードウェア
gpt-oss-20bモデルを実行するには、少なくとも20 GBのVRAMを搭載したNVIDIA GPUが必要です。

推奨GPU: NVIDIA Hopper（例：H100、H200）、NVIDIA Blackwell（例：B100、B200）、NVIDIA RTX PRO、NVIDIA RTX 50シリーズ（例：RTX 5090）。

### ソフトウェア
- CUDA Toolkit 12.8以降
- Python 3.12以降

## TensorRT-LLMのインストール

TensorRT-LLMをインストールする方法は複数あります。このガイドでは、NVIDIA NGCからの事前構築されたDockerコンテナの使用と、ソースからのビルドについて説明します。

NVIDIA Brevを使用している場合は、このセクションをスキップできます。

## NVIDIA NGCの使用

[NVIDIA NGC](https://www.nvidia.com/en-us/gpu-cloud/)からGPT-OSS用の事前構築済み[TensorRT-LLMコンテナ](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/tensorrt-llm/containers/release/tags)をプルします。
これは最も簡単に始める方法で、すべての依存関係が含まれていることを保証します。

```bash
docker pull nvcr.io/nvidia/tensorrt-llm/release:gpt-oss-dev
docker run --gpus all -it --rm -v $(pwd):/workspace nvcr.io/nvidia/tensorrt-llm/release:gpt-oss-dev
```

## Dockerの使用（ソースからビルド）

または、TensorRT-LLMコンテナをソースからビルドすることもできます。
このアプローチは、ソースコードを修正したい場合やカスタムブランチを使用したい場合に有用です。
詳細な手順については、[公式ドキュメント](https://github.com/NVIDIA/TensorRT-LLM/tree/feat/gpt-oss/docker)を参照してください。

TensorRT-LLMは間もなくpip経由で利用可能になります

> GPU アーキテクチャに関する注意: モデルを初回実行する際、TensorRT-LLM は特定の GPU アーキテクチャ（例：Hopper、Ada、Blackwell）に最適化されたエンジンを構築します。GPU の CUDA 機能（例：sm_90、sm_120）が PyTorch インストールと互換性がないという警告が表示される場合は、最新の NVIDIA ドライバーと、使用している PyTorch のバージョンに対応する CUDA Toolkit バージョンがインストールされていることを確認してください。

# TensorRT-LLM インストールの確認

In [ ]:
from tensorrt_llm import LLM, SamplingParams

# TensorRT-LLM Python APIの活用

次のコードセルでは、TensorRT-LLM Python APIを使用して以下の方法を実演します：
1. Hugging Faceから指定されたモデルの重みをダウンロードする（認証にはあなたのHF_TOKENを使用）。
2. TensorRTエンジンがまだ存在しない場合、あなたのGPUアーキテクチャ用に自動的にビルドする。
3. モデルをロードし、推論用に準備する。
4. すべてが正常に動作することを確認するため、シンプルなテキスト生成の例を実行する。

**注意**: 初回実行時は、モデルのダウンロードとエンジンのビルドのため数分かかる場合があります。
その後の実行は、エンジンがキャッシュされるため、はるかに高速になります。

In [ ]:
llm = LLM(model="openai/gpt-oss-20b")

In [ ]:
prompts = ["Hello, my name is", "The capital of France is"]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)
for output in llm.generate(prompts, sampling_params):
    print(f"Prompt: {output.prompt!r}, Generated text: {output.outputs[0].text!r}")

# まとめと次のステップ
おめでとうございます！TensorRT-LLM Python APIを使用して大規模言語モデルの最適化と実行に成功しました。

このノートブックでは、以下の方法を学びました：
- 必要な依存関係を含む環境のセットアップ
- `tensorrt_llm.LLM` APIを使用してHugging Face Hubからモデルをダウンロード
- GPUに最適化された高性能TensorRTエンジンの自動構築
- 最適化されたモデルでの推論実行


パフォーマンスと効率をさらに向上させるために、より高度な機能を探求できます：

- ベンチマーク：[ベンチマーク](https://nvidia.github.io/TensorRT-LLM/performance/performance-tuning-guide/benchmarking-default-performance.html#benchmarking-with-trtllm-bench)を実行して、TensorRT-LLMエンジンと元のHugging Faceモデルのレイテンシとスループットを比較してみてください。これは、より多くのプロンプトを反復処理し、実行時間を測定することで行えます。

- 量子化：TensorRT-LLMは、精度への影響を最小限に抑えながらモデルサイズを削減し推論を高速化するための様々な量子化技術（INT8やFP8など）を[サポート](https://github.com/NVIDIA/TensorRT-Model-Optimizer)しています。これは、リソースが制約されたハードウェアでモデルをデプロイするための強力な機能です。

- NVIDIA Dynamoでのデプロイ：本番環境では、堅牢でスケーラブル、かつマルチモデル対応のサービング機能を提供する[NVIDIA Dynamo](https://docs.nvidia.com/dynamo/latest/)を使用してTensorRT-LLMエンジンをデプロイできます。